# Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack. Example log message: INFO - haystack.utils.preprocessing - Converting data/tutorial1/218_Olenna_Tyrell.txt Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# Building a per-document query function

Rather than querying from a large document store, this notebook
explores the use case where queries need to be made against a
*specific* document. We'll build single-document document stores,
and then retrievers, readers, and pipelines as normal.

In [2]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import convert_files_to_docs
from haystack.nodes import TfidfRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

doc_dir = 'data/theringer-pdf'

docs = convert_files_to_docs(dir_path=doc_dir, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting data\theringer-pdf\robert-sarver-phoenix-suns-suspension.pdf
INFO - haystack.utils.preprocessing -  Converting data\theringer-pdf\small-market-teams-are-paying-big-prices-to-go-all-in.pdf


In [3]:
def run_query(documents, query, params=None):

    # Create in-memory document store
    document_store = InMemoryDocumentStore()
    document_store.write_documents(documents)

    # Create retriever against document store
    retriever = TfidfRetriever(document_store=document_store)

    # Load a local model or any of the QA models on
    # Hugging Face's model hub (https://huggingface.co/models)
    reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

    # Construct an extractive QA pipeline and run the query
    pipe = ExtractiveQAPipeline(reader, retriever)
    return pipe.run(query=query, params=params)

# Voilà! Ask questions!

In [4]:
# Use a util to simplify printed output
from haystack.utils import print_answers

queries = {
    'robert-sarver-phoenix-suns-suspension.pdf':
        'How long is Robert Sarver suspended for?',
    'small-market-teams-are-paying-big-prices-to-go-all-in.pdf':
        'What team was Donovan Mitchell traded to?',
}

for doc_name in queries:
    documents = [doc for doc in docs if doc.meta['name'] == doc_name]
    if not documents:
        raise ValueError
    query = queries[doc_name]
    prediction = run_query(documents, query)

    # Change `minimum` to `medium` or `all` to control the level of detail
    print_answers(prediction, details="minimum")

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.sparse -  Found 1 candidate paragraphs from 1 docs in DB
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 15 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\


Query: How long is Robert Sarver suspended for?
Answers:
[   {   'answer': 'a year',
        'context': 'and former Suns\n'
                   'employeesâ€”was to suspend Sarver from NBA activity for a '
                   'year, fine him $10 million (a fraction of his\n'
                   'net worth), and require that h'},
    {   'answer': 'one-year',
        'context': ' over an 18-year term for anyone who\n'
                   'cares to look at the sum of it.\n'
                   'A one-year suspension is nothing; a mandated course is '
                   'less than that. If the goa'},
    {   'answer': 'life',
        'context': 'hose recordings coming to light, Sterling was banned from '
                   'the league for life.\n'
                   'By June, Sarverâ€”whose repeated use of the N-word '
                   'demanded an entire d'}]


INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 15 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\   /w\   /w\   /w\   /w\   /w\   /w\   /|\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\   /'\   /'\   /'\   / \   /'\   /'\ 
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.51 Batches/s]


Query: What team was Donovan Mitchell traded to?
Answers:
[   {   'answer': 'Cavaliers',
        'context': 'amic, with small-market teams seizing the initiative. On\n'
                   'Thursday, the Cavaliers completed a stealth trade for '
                   'Donovan Mitchell, swapping three first-'},
    {   'answer': 'Utah',
        'context': 'd remains together for years to come, the unprotected '
                   'picks\n'
                   'they sent to Utah wouldn’t be as valuable; the Pelicans '
                   'probably won’t land any prized pic'},
    {   'answer': 'Cleveland',
        'context': 'hree teams settled into the play-in\n'
                   'range in the middle of the league.\n'
                   'Cleveland’s hope is clearly for Mitchell, who will '
                   'celebrate his 26th birthday '}]
